# Predicting Housing Data
This notebook makes all the necessary transformations to the data prior to training the model.

## Import Libraries

In [1]:
# DML
import pandas as pd

# Encoders
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

# Convenience functions
from my_functions import fill_all_missing_values

## Load data

In [2]:
# Read training dataset
df_train = pd.read_csv(
    filepath_or_buffer='../../data/raw/train.csv'
)

# Read testing dataset
df_test = pd.read_csv(
    filepath_or_buffer='../../data/raw/test.csv'
)

## Preprocessing
Fill missing values

In [3]:
# Columns to fill (manually)
cols_fillna = [
    'FireplaceQu',
    'BsmtQual',
    'BsmtCond',
    'BsmtFinType1',
    'BsmtFinType2'
]

# Fill NaNs manually
df_train[cols_fillna] = df_train[cols_fillna].fillna('NA')
df_test[cols_fillna] = df_test[cols_fillna].fillna('NA')

# Fill columns with convenience function
fill_all_missing_values(df_train)
fill_all_missing_values(df_test)

Transform ordinal columns

In [4]:
# List of ordinal columns
cols_ordinal = [
    'BsmtQual',
    'BsmtCond',
    'BsmtQual',
    'ExterCond',
    'ExterQual',
    'KitchenQual',
    'PavedDrive',
    'Electrical',
    'BsmtFinType1',
    'BsmtFinType2',
    'Utilities',
    'MSZoning',
    'Foundation',
    'Neighborhood',
    'MasVnrType',
    'SaleCondition',
    'RoofStyle',
    'RoofMatl'
]

# Transform all columns
for col in cols_ordinal:
    oe = OrdinalEncoder()
    df_train[col] = oe.fit_transform(df_train[[col]])
    df_test[col] = oe.transform(df_test[[col]])
    del oe

Transform categorical columns

In [5]:
cols_level = [
    'Street',
    'BldgType',
    'SaleType',
    'CentralAir'
]

for col in cols_level:
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col])
    df_test[col] = le.transform(df_test[col])

## Feature Engineering

In [6]:
# Create on both dfs
for obj in [df_train, df_test]:
    obj['BsmtRating'] = obj['BsmtCond'] * obj['BsmtQual']
    obj['ExterRating'] = obj['ExterCond'] * obj['ExterQual']
    obj['BsmtFinTypeRating'] = obj['BsmtFinType1'] * obj['BsmtFinType2']
    obj['BsmtBath'] = obj['BsmtFullBath'] + obj['BsmtHalfBath']
    obj['Bath'] = obj['FullBath'] + obj['HalfBath']
    obj['PorchArea'] = obj['OpenPorchSF'] + obj['EnclosedPorch'] + obj['3SsnPorch'] + obj['ScreenPorch']

Drop useless columns

In [7]:
# List of columns to drop
cols_drop = [
    'Id',
    'Alley',
    'PoolQC',
    'MiscFeature',
    'Fence',
    'MoSold',
    'YrSold',
    'MSSubClass',
    'GarageType',
    'GarageArea',
    'GarageYrBlt',
    'GarageFinish',
    'YearRemodAdd',
    'LandSlope',
    'BsmtUnfSF',
    'BsmtExposure',
    '2ndFlrSF',
    'LowQualFinSF',
    'Condition1',
    'Condition2',
    'Heating',
    'Exterior1st',
    'Exterior2nd',
    'HouseStyle',
    'LotShape',
    'LandContour',
    'LotConfig',
    'Functional',
    'BsmtFinSF1',
    'BsmtFinSF2',
    'FireplaceQu',
    'WoodDeckSF',
    'GarageQual',
    'GarageCond',
    'OverallCond',
    'OverallQual',
    'ExterCond',
    'ExterQual',
    'BsmtCond',
    'BsmtQual',
    'BsmtFinType1',
    'BsmtFinType2',
    'HeatingQC',
    'OpenPorchSF',
    'EnclosedPorch',
    '3SsnPorch',
    'ScreenPorch',
    'BsmtFullBath',
    'BsmtHalfBath',
    'FullBath',
    'HalfBath'
]

# Drop columns on both dfs
for obj in [df_train, df_test]:
    obj.drop(
        labels=cols_drop,
        axis=1,
        inplace=True
    )

## Export processed data

In [9]:
# Export train
df_train.to_csv(
    path_or_buf='../../data/processed/train.csv',
    index=False
)

# Export test
df_test.to_csv(
    path_or_buf='../../data/processed/test.csv',
    index=False
)

## Model

In [ ]:
y = df_train['SalePrice']
X = df_train.drop(['SalePrice'], axis=1)

candidate_max_leaf_nodes = [250]
#model = LinearRegression()

for node in candidate_max_leaf_nodes:
    model = RandomForestRegressor(max_leaf_nodes=node,)
    model.fit(X, y)
    score = cross_val_score(model, X, y, cv=10)
    print(score.mean())

## Final output

In [ ]:
price = model.predict(df_test)
submission = pd.DataFrame({
    "Id": test_ids,
    "SalePrice": price
})

submission.to_csv("submission.csv", index=False)
submission.sample(10)

In [ ]:
print("end")